# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 114 new papers today
          42 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/41 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.05718


extracting tarball to tmp_2301.05718...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.05718/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'fluxes.tex' from 'tmp_2301.05718/fluxes.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'defs' from 'tmp_2301.05718/defs.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


...

extracting tarball to tmp_2301.06947... done.
Retrieving document from  https://arxiv.org/e-print/2301.07029


extracting tarball to tmp_2301.07029...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.07044


extracting tarball to tmp_2301.07044... done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3442: LatexWarning: Found documentclass in tmp_2301.07044/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05718-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05718) | **Serendipitous Nebular-phase JWST Imaging of SN Ia 2021aefx: Testing the  Confinement of 56-Co Decay Energy**  |
|| Ness Mayker Chen, et al. -- incl., <mark>Lindsey Kwok</mark>, <mark>Frank Bigiel</mark>, <mark>Francesca Pinna</mark>, <mark>Eva Schinnerer</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| *Submitted to ApJL; 11 pages, 4 figures, 2 tables in two-column AASTEX63 format. Comments welcome*|
|**Abstract**| We present new 0.3-21 micron photometry of SN 2021aefx in the spiral galaxy NGC 1566 at +357 days after B-band maximum, including the first detection of any SN Ia at >15 micron. These observations follow earlier JWST observations of SN 2021aefx at +255 days after the time of maximum brightness, allowing us to probe the temporal evolution of the emission properties. We measure the fraction of flux emerging at different wavelengths and its temporal evolution. Additionally, the integrated 0.3-14 micron decay rate of $\Delta m_{0.3-14} = 1.35 \pm 0.05$ mag/100 days is higher than the decline rate from the radioactive decay of $^{56}$Co of $\sim 1.2$mag/100 days. The most plausible explanation for this discrepancy is that flux is shifting to >14 micron, and future JWST observations of SNe Ia will be able to directly test this hypothesis. However, models predicting non-radiative energy loss cannot be excluded with the present data. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.05720-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.05720) | **KAOSS: turbulent, but disc-like kinematics in dust-obscured star-forming  galaxies at $z\sim$1.3-2.6**  |
|| Jack E. Birkin, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| *20 pages, 12 figures, submitted to MNRAS*|
|**Abstract**| We present spatially resolved kinematics of 31 ALMA-identified dust-obscured star-forming galaxies (DSFGs) at $z\sim$1.3-2.6, as traced by H$\alpha$ emission using VLT/KMOS near-infrared integral field spectroscopy from our on-going Large Programme ''KMOS-ALMA Observations of Submillimetre Sources'' (KAOSS). We derive H$\alpha$ rotation curves and velocity dispersion profiles for the DSFGs. Of the 31 sources with bright, spatially extended H$\alpha$ emission, 25 display rotation curves that are well fit by a Freeman disc model, enabling us to measure a median inclination-corrected velocity at 2.2$R_{\rm d}$ of $v_{\rm rot}$ = 190 $\pm$ 30 kms$^{-1}$ and a median intrinsic velocity dispersion of $\sigma_0$ = 87 $\pm$ 6 kms$^{-1}$ for these $\textit{disc-like}$ sources. By comparison with less actively star-forming galaxies, KAOSS DSFGs are both faster rotating and more turbulent, but have similar $v_{\rm rot}/\sigma_0$ ratios, median 2.4 $\pm$ 0.5. We suggest that $v_{\rm rot}/\sigma_0$ alone is insufficient to describe the kinematics of DSFGs, which are not kinematically ''cold'' discs, and that the individual components $v_{\rm rot}$ and $\sigma_0$ indicate that they are in fact turbulent, but rotationally supported systems in $\sim$50 per cent of cases. This turbulence may be driven by star formation or mergers/interactions. We estimate the normalisation of the stellar Tully-Fisher relation (sTFR) for the disc-like DSFGs and compare it with local studies, finding no evolution at fixed slope between $z\sim$2 and $z\sim$0. Finally, we use kinematic estimates of DSFG halo masses to investigate the stellar-to-halo mass relation, finding our sources to be consistent with shock heating and strong feedback which likely drives the declining stellar content in the most massive halos. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.06575-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.06575) | **Neural posterior estimation for exoplanetary atmospheric retrieval**  |
|| Malavika Vasist, et al. -- incl., <mark>Paul Mollière</mark>, <mark>Evert Nasedkin</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| **|
|**Abstract**| Retrieving the physical parameters from spectroscopic observations of exoplanets is key to understanding their atmospheric properties. Exoplanetary atmospheric retrievals are usually based on approximate Bayesian inference and rely on sampling-based approaches to compute parameter posterior distributions. Accurate or repeated retrievals, however, can result in very long computation times due to the sequential nature of sampling-based algorithms. We aim to amortize exoplanetary atmospheric retrieval using neural posterior estimation (NPE), a simulation-based inference algorithm based on variational inference and normalizing flows. In this way, we aim (i) to strongly reduce inference time, (ii) to scale inference to complex simulation models with many nuisance parameters or intractable likelihood functions, and (iii) to enable the statistical validation of the inference results. We evaluate NPE on a radiative transfer model for exoplanet spectra petitRADTRANS, including the effects of scattering and clouds. We train a neural autoregressive flow to quickly estimate posteriors and compare against retrievals computed with MultiNest. NPE produces accurate posterior approximations while reducing inference time down to a few seconds. We demonstrate the computational faithfulness of our posterior approximations using inference diagnostics including posterior predictive checks and coverage, taking advantage of the quasi-instantaneous inference time of NPE. Our analysis confirms the reliability of the approximate posteriors produced by NPE. The accuracy and reliability of the inference results produced by NPE establishes it as a promising approach for atmospheric retrievals. Amortization of the posterior inference makes repeated inference on several observations computationally inexpensive since it does not require on-the-fly simulations, making the retrieval efficient, scalable, and testable. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.06819-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.06819) | **Massive pre-main-sequence stars in M17 -- Modelling hydrogen and dust in  MYSO disks**  |
|| <mark>Frank Backs</mark>, et al. |
|*Appeared on*| *2023-01-18*|
|*Comments*| *18 pages, 16 figures. Accepted for publication in A&A*|
|**Abstract**| The young massive-star-forming region M17 contains optically visible massive pre-main-sequence stars that are surrounded by circumstellar disks. Such disks are expected to disappear when these stars reach the main sequence. The physical and dynamical structure of these remnant disks are poorly constrained, especially the inner regions where accretion, photo-evaporation, and companion formation and migration may be ongoing. We aim to constrain the physical properties of the inner parts of the circumstellar disks of massive young stellar objects B243 (6 Msun) and B331 (12 Msun), two systems for which the central star has been detected and characterized previously despite strong dust extinction. Two-dimensional radiation thermo-chemical modelling with ProDiMo of double-peaked hydrogen lines of the Paschen and Brackett series observed with X-shooter was used to probe the properties of the inner disks. Additionally, the dust structure was studied by fitting the optical and near-infrared spectral energy distribution. B243 features a hot gaseous inner disk with dust at the sublimation radius at 3 AU. The disk appears truncated at roughly 6.5 AU; a cool outer disk of gas and dust may be present, but it cannot be detected with our data. B331 also has a hot gaseous inner disk. A gap separates the inner disk from a colder dusty outer disk starting at up to 100 AU. In both sources the inner disk extends to almost the stellar surface. Chemistry is essential for the ionization of hydrogen in these disks. The lack of a gap between the central objects and these disks suggests that they accrete through boundary-layer accretion. This would exclude the stars having a strong magnetic field. Their structures suggest that both disks are transitional in nature, that is to say they are in the process of being cleared, either through boundary-layer accretion, photo-evaporation, or through companion activity. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.07044-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.07044) | **Constraining uber gravity with recent observations and elucidating the  $H_0$ problem**  |
|| Gustavo A. Concha Valdez, et al. -- incl., <mark>A. Hernández-Almada</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| *9 pages, 4 figures*|
|**Abstract**| This paper studies both $\Lambda$CDM and CDM models under the \"uber gravity theory, named \"u$\Lambda$CDM and \"uCDM respectively. We report bounds over their parameter phase-space using several cosmological data, in particular, the recent Pantheon+ sample. Based on the joint analysis, the best fit value of the \"uber characteristic parameter is $z_\oplus = 0.028^{+0.036}_{-0.020}$ and $z_\oplus = 0.960^{+0.031}_{-0.030}$ at 68\% confidence level for \"u$\Lambda$CDM and \"uCDM respectively. Furthermore, we find that the $\mathbb{H}0(z)$ diagnostic suggests the $H_0$ tension is not alleviated. Finally, both models are statistically compared with $\Lambda$CDM through the Akaike and Bayesian information criteria, which suggest that there is a modest evidence against for \"u$\Lambda$CDM, a strongest evidence for \"uCDM, against for the joint analysis, but both \"uber gravity models and $\Lambda$CDM are equally preferred for most of the single samples. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.06253-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.06253) | **Switching integrators reversibly in the astrophysical $N$-body problem**  |
|| <mark>David M. Hernandez</mark>, <mark>Walter Dehnen</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| *10 pages, 8 figures, submitted to MNRAS, comments welcome*|
|**Abstract**| We present a simple algorithm to switch between $N$-body time integrators in a reversible way. We apply it to planetary systems undergoing arbitrarily close encounters and highly eccentric orbits, but the potential applications are broader. Upgrading an ordinary non-reversible switching integrator to a reversible one is straightforward and introduces no appreciable computational burden in our tests. Our method checks if the integrator during the time step violates a time-symmetric selection condition and redoes the step if necessary. In our experiments a few percent of steps would have violated the condition without our corrections. By eliminating them the algorithm avoids long-term error accumulation, of several orders magnitude in some cases. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.06350-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.06350) | **A First Look at the JWST MIRI/LRS Phase Curve of WASP-43b**  |
|| Taylor J. Bell, et al. -- incl., <mark>Laura Kreidberg</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| **|
|**Abstract**| We observed a full-orbit phase curve of the hot Jupiter WASP-43b with MIRI/LRS as part of the Transiting Exoplanet Community Early Release Science Program. Here we report preliminary findings for the instrument performance from the team's MIRI Working Group. Overall we find that MIRI's performance for phase curve observations is excellent, with a few minor caveats. The key takeaways for Cycle 2 planning with MIRI/LRS are: (1) long-duration observations (> 24 hours) have now been successfully executed; (2) for phase curves, we recommend including a one-hour burn-in period prior to taking science data to mitigate the effects of the ramp systematic; and (3) we do not yet recommend partial phase curve observations. In addition, we also find that: the position of the spectrum on the detector is stable to within 0.03 pixels over the full 26.5-hour observation; the light curves typically show a systematic downward ramp that is strongest for the first 30 minutes, but continues to decay for hours; from 10.6-11.8 microns, the ramp effect has remarkably different behavior, possibly due to a different illumination history for the affected region of the detector; after trimming the integrations most affected by the initial ramps and correcting the remaining systematics with analytic models, we obtain residuals to the light-curve fits that are typically within 25% of the photon noise limit for 0.5-micron spectroscopic bins; non-linearity correction is not a significant source of additional noise for WASP-43, though it may be an issue for brighter targets; the gain value of 5.5 electrons/DN currently on CRDS and JDox is known to be incorrect, and the current best estimate for the gain is approximately 3.1 electrons/DN; new reference files for the JWST calibration pipeline reflecting these findings are under development at STScI. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.06709-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.06709) | **Satellite observations of the Alfvénic Transition from Weak to Strong  Magnetohydrodynamic Turbulence**  |
|| <mark>Siqi Zhao</mark>, et al. -- incl., <mark>Terry Z. Liu</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| *submitted; 24 pages; 4 figures*|
|**Abstract**| Plasma turbulence is a ubiquitous dynamical process that transfers energy across many spatial and temporal scales in astrophysical and space plasma systems. Although the theory of anisotropic magnetohydrodynamic (MHD) turbulence has successfully described phenomena in nature, its core prediction of an Alfvenic transition from weak to strong MHD turbulence when energy cascades from large to small scales has not been observationally confirmed. Here we report the first observational evidence for the Alfvenic weak-to-strong transition in MHD turbulence in the terrestrial magnetosheath using the four Cluster spacecraft. The observed transition indicates the universal existence of strong turbulence regardless of the initial level of MHD fluctuations. Moreover, the observations demonstrate that the nonlinear interactions of MHD turbulence play a crucial role in the energy cascade, widening the directions of the energy cascade and broadening the fluctuating frequencies. Our work takes a critical step toward understanding the complete picture of turbulence cascade, connecting the weak and strong MHD turbulence systems. It will have broad implications in star formation, energetic particle transport, turbulent dynamo, and solar corona or solar wind heating. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.06779-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.06779) | **KMT-2022-BLG-0440Lb: A New $q < 10^{-4}$ Microlensing Planet with the  Central-Resonant Caustic Degeneracy Broken**  |
|| <mark>Jiyuan Zhang</mark>, et al. -- incl., <mark>Andrew Gould</mark>, <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark>, <mark>Zhuokai Liu</mark> |
|*Appeared on*| *2023-01-18*|
|*Comments*| *to be submitted to MNRAS*|
|**Abstract**| We present the observations and analysis of a high-magnification microlensing planetary event, KMT-2022-BLG-0440, for which the weak and short-lived planetary signal was covered by both the KMTNet survey and follow-up observations. The binary-lens models with a central caustic provide the best fits, with a planet/host mass ratio, $q = 0.75$--$1.00 \times 10^{-4}$ at $1\sigma$. The binary-lens models with a resonant caustic and a brown-dwarf mass ratio are both excluded by $\Delta\chi^2 > 70$. The binary-source model can fit the anomaly well but is rejected by the ``color argument'' on the second source. From Bayesian analyses, it is estimated that the host star is likely a K or M dwarf located in the Galactic disk, the planet probably has a Neptune-mass, and the projected planet-host separation is $1.9^{+0.6}_{-0.7}$ or $4.6^{+1.4}_{-1.7}$ au, subject to the close/wide degeneracy. This is the third $q < 10^{-4}$ planet from a high-magnification planetary signal ($A \gtrsim 65$). Together with another such planet, KMT-2021-BLG-0171Lb, the ongoing follow-up program for the KMTNet high-magnification events has demonstrated its ability in detecting high-magnification planetary signals for $q < 10^{-4}$ planets, which are challenging for the current microlensing surveys. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.05718.md
    + _build/html/tmp_2301.05718/./SN2021aefx.png
    + _build/html/tmp_2301.05718/./SED.png
    + _build/html/tmp_2301.05718/./nebdec.png
exported in  _build/html/2301.05720.md
    + _build/html/tmp_2301.05720/./figures/K_S870.png
    + _build/html/tmp_2301.05720/./figures/SFR_Mstar.png
    + _build/html/tmp_2301.05720/./figures/kaoss_vel_fields_sigma_01_Ha.png
    + _build/html/tmp_2301.05720/./figures/kaoss_rot_curves_vel_fields_01_Ha.png
exported in  _build/html/2301.06575.md
    + _build/html/tmp_2301.06575/./plots/fig1.png
    + _build/html/tmp_2301.06575/./plots/consistency_wlfixed.png
    + _build/html/tmp_2301.06575/./plots/output-figure0.png
exported in  _build/html/2301.06819.md
    + _build/html/tmp_2301.06819/./parameters_per_line.png
    + _build/html/tmp_2301.06819/./B243_line_profile_overview.png
    + _build/html/tmp_2301.06819/./B331_line_profile_overview.png
exported in  _build/html/2301.06873.md
    + _build/html/tmp_2301.06873

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\head}[2]{\multicolumn{1}{>{\centering\arraybackslash}p{#1}}{\textbf{#2}}}$
$\newcommand{\name}{SN~2021aefx\xspace}$
$\newcommand{\mat}[1]{\textcolor{purple}{MAT: #1}}$
$\newcommand{\CITE}{\textcolor{orange}{(add citation))}\xspace}$
$\newcommand{\um}{\hbox{\ensuremath{\mu\rm m}}\xspace}$
$\newcommand{\jwst}{\emph{JWST}\xspace}$
$\newcommand{\sneia}{SNe Ia\xspace}$
$\newcommand{\snia}{SN Ia\xspace}$
$\newcommand{\tmax}{\hbox{\ensuremath{t_{\rm{max}}}}\xspace}$
$\newcommand{\fopt}{\hbox{\ensuremath{F_{\rm{opt}}}}\xspace}$
$\newcommand{\fmir}{\hbox{\ensuremath{F_{\rm{mIR}}}}\xspace}$
$\newcommand{\lak}[1]{\textcolor{maroon}{LAK: #1}}$
$\newcommand{\OSU}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{\CCAPP}{\affil{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{\NRAO}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{\UAlberta}{\affil{4-183 CCIS, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}}$
$\newcommand{\Rutgers}{\affiliation{Department of Physics and Astronomy, Rutgers, the State University of New Jersey,\\136 Frelinghuysen Road, Piscataway, NJ 08854, USA}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\Alberta}{\affil{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{\HD}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{\ITA}{\affiliation{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{\IWR}{\affiliation{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{\ANU}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\ASTROThreeD}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{\MPIA}{\affiliation{Max Planck Institute for Astronomy, K\"onigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\MISSING}{\affiliation{\textcolor{purple}{Need affiliation}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\head$}[2]{\multicolumn{1}{>{\centering\arraybackslash}p{#1}}{\textbf{#2}}}$
$\newcommand{$\name$}{SN~2021aefx$\xspace$}$
$\newcommand{$\mat$}[1]{\textcolor{purple}{MAT: #1}}$
$\newcommand{$\CITE$}{\textcolor{orange}{(add citation))}$\xspace$}$
$\newcommand{$\um$}{\hbox{$\ensuremath${\mu\rm m}}$\xspace$}$
$\newcommand{$\jwst$}{\emph{JWST}$\xspace$}$
$\newcommand{$\sneia$}{SNe Ia$\xspace$}$
$\newcommand{$\snia$}{SN Ia$\xspace$}$
$\newcommand{$\tmax$}{\hbox{$\ensuremath${t_{\rm{max}}}}$\xspace$}$
$\newcommand{$\fopt$}{\hbox{$\ensuremath${F_{\rm{opt}}}}$\xspace$}$
$\newcommand{$\fmir$}{\hbox{$\ensuremath${F_{\rm{mIR}}}}$\xspace$}$
$\newcommand{$\lak$}[1]{\textcolor{maroon}{LAK: #1}}$
$\newcommand{$\OSU$}{\affil{Department of Astronomy, The Ohio State University, 140 West 18th Avenue, Columbus, Ohio 43210, USA}}$
$\newcommand{$\CCAPP$}{\affil{Center for Cosmology and Astroparticle Physics, 191 West Woodruff Avenue, Columbus, OH 43210, USA}}$
$\newcommand{$\NRAO$}{\affil{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903-2475, USA}}$
$\newcommand{$\UAlberta$}{\affil{4-183 CCIS, University of Alberta, Edmonton, Alberta, T6G 2E1, Canada}}$
$\newcommand{$\Rutgers$}{\affiliation{Department of Physics and Astronomy, Rutgers, the State University of New Jersey,\\136 Frelinghuysen Road, Piscataway, NJ 08854, USA}}$
$\newcommand{$\STScI$}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{$\Alberta$}{\affil{Department of Physics, University of Alberta, Edmonton, AB T6G 2E1, Canada}}$
$\newcommand{$\HD$}{\affil{Astronomisches Rechen-Institut, Zentrum f\"{u}r Astronomie der Universit\"{a}t Heidelberg, M\"{o}nchhofstra\ss e 12-14, D-69120 Heidelberg, Germany}}$
$\newcommand{$\ITA$}{\affiliation{Universit\"{a}t Heidelberg, Zentrum f\"{u}r Astronomie, Institut f\"{u}r Theoretische Astrophysik, Albert-Ueberle-Str 2, D-69120 Heidelberg, Germany}}$
$\newcommand{$\IWR$}{\affiliation{Universit\"{a}t Heidelberg, Interdisziplin\"{a}res Zentrum f\"{u}r Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, D-69120 Heidelberg, Germany}}$
$\newcommand{$\ANU$}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{$\ASTROThreeD$}{\affiliation{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}}$
$\newcommand{$\MPIA$}{\affiliation{Max Planck Institute for Astronomy, K\"onigstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{$\MISSING$}{\affiliation{\textcolor{purple}{Need affiliation}}}$</div>



<div id="title">

# Co Decay Energy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.05718-b31b1b.svg)](https://arxiv.org/abs/2301.05718)<mark>Appeared on: 2023-01-18</mark> - _Submitted to ApJL; 11 pages, 4 figures, 2 tables in two-column AASTEX63 format. Comments welcome_

</div>
<div id="authors">

Ness Mayker~Chen, et al. -- incl., <mark>Lindsey Kwok</mark>, <mark>Frank Bigiel</mark>, <mark>Francesca~Pinna</mark>, <mark>Eva Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We present new$0.3-21 $\um$$photometry of$\name$in the spiral galaxy NGC 1566 at$+357$days after$B$-band maximum, including the first detection of any SN Ia at$>15 \micron$. These observations follow earlier$\jwst$observations of$\name$at$+255$days after the time of maximum brightness, allowing us to probe the temporal evolution of the emission properties. We measure the fraction of flux emerging at different wavelengths and its temporal evolution. Additionally, the integrated$0.3-14 $\um$$decay rate of$\Delta m_{0.3-14} = 1.35 \pm 0.05 $mag/100 days is higher than the decline rate from the radioactive decay of$^{56}$Co of$\sim 1.2$mag/100 days. The most plausible explanation for this discrepancy is that flux is shifting to$>14 $\um$$, and future$\jwst$observations of$\sneia$will be able to directly test this hypothesis. However, models predicting non-radiative energy loss cannot be excluded with the present data.

</div>

<div id="div_fig1">

<img src="tmp_2301.05718/./SN2021aefx.png" alt="Fig3" width="100%"/>

**Figure 3. -** \textit{$\name$ in NGC 1566 at $\approx 2{-}21\mu$m.} Left panel: MIRI \emph{F1130W} PHANGS-JWST image of NGC 1566 showing the location of $\name$ marked with a green circle. Right panels: zoom-ins on $\name$ in each PHANGS-JWST filter. The top four panels show 200 pc $\times $200 pc cutouts from NIRCam images at 2.0$-$3.6 $\micron$. The bottom four panels show $1$ kpc $\times$ $1$ kpc MIRI images at $7.7-21  \micron$. The inner green circle marks the aperture used in the photometry measurement, and the two concentric, dashed, cyan circles mark the inner and outer apertures used for the background subtraction. The results of the photometry appear in Table \ref{tab:photometry}. \label{fig:SN2021aefx} (*fig:SN2021aefx*)

</div>
<div id="div_fig2">

<img src="tmp_2301.05718/./SED.png" alt="Fig4" width="100%"/>

**Figure 4. -** \textit{A comparison of the Near- and Mid- IR flux of $\name$ at +255 and +357 days.} Purple stars mark our flux measurements from each filter and  black open circles signify synthetic photometry from the +255-day spectrum (black line). Horizontal error bars show the nominal wavelength width of each filter. Vertical error bars mark the uncertainty in the measurement and are shown, but are typically smaller than the points. The vertical grey dashed lines mark emission features identified by \cite{Kwok2022}. NIRCam and MIRI filter curves are plotted along the bottom axis.
\label{fig:SED} (*fig:SED*)

</div>
<div id="div_fig3">

<img src="tmp_2301.05718/./nebdec.png" alt="Fig5" width="100%"/>

**Figure 5. -** Nebular-phase decline rates as a function of wavelength for $\name$. The green region represents the $J$- and $H$-band decay rates of $0.0-0.3$ mag/100 days from \citet{Graur2020}. Blue horizontal lines show the energy deposition rate for $^{56}$Co with (dashed) and without (dotted) including the $\gamma$-ray contribution. The black line and shaded region represent the integrated $0.3-14 $\um$$ decline rate derived in \S\ref{sec:Results}. Data for this figure are included in the online version of the manuscript. (*fig:nebdec*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\todo}[1]{{\color{red} ZZZ: #1}}$
$\newcommand{\thefigure}{\arabic{figure} (Cont.)}$
$\newcommand{\thefigure}{\arabic{figure}}$
$\newcommand{\thefigure}{A1 (Cont.)}$
$\newcommand{\thefigure}{\arabic{figure}}$
$\newcommand{\thefigure}{A1 (Cont.)}$
$\newcommand{\thefigure}{\arabic{figure}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\todo$}[1]{{\color{red} ZZZ: #1}}$
$\newcommand{$\thefigure$}{\arabic{figure} (Cont.)}$
$\newcommand{$\thefigure$}{\arabic{figure}}$
$\newcommand{$\thefigure$}{A1 (Cont.)}$
$\newcommand{$\thefigure$}{\arabic{figure}}$
$\newcommand{$\thefigure$}{A1 (Cont.)}$
$\newcommand{$\thefigure$}{\arabic{figure}}$</div>



<div id="title">

#  1.3--2.6

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.05720-b31b1b.svg)](https://arxiv.org/abs/2301.05720)<mark>Appeared on: 2023-01-18</mark> - _20 pages, 12 figures, submitted to MNRAS_

</div>
<div id="authors">

Jack E. Birkin, et al. -- incl., <mark><mark>E. Schinnerer</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present spatially resolved kinematics of 31 ALMA-identified dust-obscured star-forming galaxies (DSFGs) at$z$$\sim$1.3--2.6, as traced by H$\alpha$emission using VLT/KMOS near-infrared integral field spectroscopy from our on-going Large Programme "KMOS-ALMA Observations of Submillimetre Sources" (KAOSS). We derive H$\alpha$rotation curves and velocity dispersion profiles for the DSFGs. Of the 31 sources with bright, spatially extended H$\alpha$emission, 25 display rotation curves that are well fit by a Freeman disc model, enabling us to measure a median inclination-corrected velocity at 2.2$R_{\rm d}$of$v_{\rm rot}$$=$190$\pm$30 km s$^{-1}$and a median intrinsic velocity dispersion of$\sigma_0$$=$87$\pm$6 km s$^{-1}$for these{\it disc-like}sources. By comparison with less actively star-forming galaxies, KAOSS DSFGs are both faster rotating and more turbulent, but have similar$v_{\rm rot}/\sigma_0$ratios, median 2.4$\pm$0.5. We suggest that$v_{\rm rot}/\sigma_0$alone is insufficient to describe the kinematics of DSFGs, which are not kinematically "cold" discs, and that the individual components$v_{\rm rot}$and$\sigma_0$indicate that they are in fact turbulent, but rotationally supported systems in$\sim$50 per cent of cases. This turbulence may be driven by star formation or mergers/interactions. We estimate the normalisation of the stellar Tully-Fisher relation (sTFR) for the disc-like DSFGs and compare it with local studies, finding no evolution at fixed slope between$z$$\sim$2 and$z$$\sim$0. Finally, we use kinematic estimates of DSFG halo masses to investigate the stellar-to-halo mass relation, finding our sources to be consistent with shock heating and strong feedback which likely drives the declining stellar content in the most massive halos.

</div>

<div id="div_fig1">

<img src="tmp_2301.05720/./figures/K_S870.png" alt="Fig5.1" width="50%"/><img src="tmp_2301.05720/./figures/SFR_Mstar.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** {\bf a)}$K$-band magnitude versus 870-$\mu$m flux density ($S_{870}$) for the KAOSS resolved sample compared with the 707 AS2UDS DSFGs \protect\citep{dudzeviciute20} and $z$$\sim$ 1.5 star-forming galaxies from KGES \protect\citep{tiley21}. The KAOSS resolved sample is generally representative of the range of 870-$\mu$m flux densities in the DSFG population, but biased towards near-infrared-brighter sources. For the KGES sample we indicate the region containing the $K$-band magnitudes of the sample and the expected 870-$\mu$m flux densities estimated from SED-derived dust masses. The KAOSS sample is complemented by KGES in that the latter extends the range in SFR by an order of magnitude. {\bf b)} Star-formation rate versus $M_\ast$ for the same samples, now also including six $z$$\sim$ 2.5 ULIRGs from \protect\cite{hogan21}. KAOSS probes sources that are approximately an order of magnitude more massive than KGES, and overlaps with the parameter range of the {\it Herschel}-selected ULIRGs in \protect\cite{hogan21}. Also shown here are $z$$\sim$ 1.3--2.6 UV/optically selected galaxies from the SINS survey \protect\citep{forster-schreiber09}, which has some overlap with KAOSS, but also probes lower-mass and lower-SFR systems. (*fig:kaoss_selection*)

</div>
<div id="div_fig2">

<img src="tmp_2301.05720/./figures/kaoss_vel_fields_sigma_01_Ha.png" alt="Fig6" width="100%"/>

**Figure 6. -** {\it HST} F160W or ground-based $K$-band images (left), velocity fields (middle) and velocity dispersion profiles (right) for our sample of 31 resolved KAOSS DSFGs. The images are 7.5$"$$\times$ 7.5$"$ in size, and the white box indicates the region shown in the velocity and velocity dispersion maps (3.2$"$$\times$ 3.2$"$ in size). Sources are ordered by the S/N of the H$\alpha$ emission line (shown in the bottom-left corner of the middle panels) and we indicate the inclination angle derived for the source from {\sc galfit} modelling. The black lines on the centre and right panels indicate the kinematic position angle along which rotation curves are measured. For the left panels we indicate the three filters that make up the RGB colour image, or the single filter in cases where the image is greyscale. (*fig:kaoss_vel_fields_1*)

</div>
<div id="div_fig3">

<img src="tmp_2301.05720/./figures/kaoss_rot_curves_vel_fields_01_Ha.png" alt="Fig13" width="100%"/>

**Figure 13. -** H$\alpha$ rotation curves (left) and velocity dispersion profiles (right) for the resolved KAOSS sample. Sources are ordered by the S/N of the H$\alpha$ emission line as in Fig. \ref{fig:kaoss_vel_fields_1}. The solid black lines indicate Freeman disc model fits to the data, the vertical dashed lines indicate $\pm$ 2.2 times the disc radius (thin) and $\pm$ 2.2 times the disc radius convolved with $\sigma_{\rm PSF}$(thick), the latter at which we measure the rotation velocity, and the cross indicates the original source centroid. Points that are plotted as open circles are masked in the fitting procedure. For the dispersion profiles we indicate the observed velocity dispersion, $\sigma_{\rm obs}$(horizontal black solid line), along with the method used to measure it (top right), and the beam smearing-corrected intrinsic velocity dispersion (horizontal black dashed line). We also indicate the method used to centre the velocity field in the top right corner of the dispersion profile panels. (*fig:kaoss_rot_curves_1*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\knowing}{|}$
$\newcommand{\}{hyper}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\knowing$}{|}$
$\newcommand{\}{hyper}$</div>



<div id="title">

# Neural posterior estimation for exoplanetary atmospheric retrieval

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.06575-b31b1b.svg)](https://arxiv.org/abs/2301.06575)<mark>Appeared on: 2023-01-18</mark> - __

</div>
<div id="authors">

Malavika Vasist, et al. -- incl., <mark>Paul Mollière</mark>, <mark>Evert Nasedkin</mark>

</div>
<div id="abstract">

**Abstract:** Retrieving the physical parameters from spectroscopic observations of exoplanets is key to understanding their atmospheric properties.Exoplanetary atmospheric retrievals are usually based on approximate Bayesian inference and rely on sampling-based approaches to compute parameter posterior distributions.Accurate or repeated retrievals, however, can result in very long computation times due to the sequential nature of sampling-based algorithms.We aim to amortize exoplanetary atmospheric retrieval using neural posterior estimation (NPE), a simulation-based inference algorithm based on variational inference and normalizing flows.In this way, we aim (i) to strongly reduce inference time, (ii) to scale inference to complex simulation models with many nuisance parameters or intractable likelihood functions, and (iii) to enable the statistical validation of the inference results.We evaluate NPE on a radiative transfer model for exoplanet spectra (\texttt{petitRADTRANS}), including the effects of scattering and clouds.We train a neural autoregressive flow to quickly estimate posteriors and compare against retrievals computed with\texttt{MultiNest}.NPE produces accurate posterior approximations while reducing inference time down to a few seconds.We demonstrate the computational faithfulness of our posterior approximations using inference diagnostics including posterior predictive checks and coverage, taking advantage of the quasi-instantaneous inference time of NPE. Our analysis confirms the reliability of the approximate posteriors produced by NPE.The inference results produced by NPE appear to be accurate and reliable, establishing this algorithm as a promising approach for atmospheric retrieval. Its main benefits come from the amortization of posterior inference: once trained, inference does not require on-the-fly simulations and can be repeated several times for many observations at very low computational cost. This enables efficient, scalable, and testable atmospheric retrieval.

</div>

<div id="div_fig1">

<img src="tmp_2301.06575/./plots/fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** Inference pipeline using amortized neural posterior estimation. The joint simulation model $p(x,\theta)=p(\theta)p(x|\theta)$ is used to generate a training set $\{ (\theta, x) \}$ of model parameters $\theta$ and exoplanet spectra observations $x$. A conditional normalizing flow $p_\phi(\theta|x)$ composed of an embedding network and three invertible transformations $t_i$ is trained to estimate the posterior density $p(\theta|x)$. Once trained, sampling from the posterior estimator is as fast as a forward pass through the normalizing flow. Inference can be repeated for many observations without having to regenerate data nor retrain the normalizing flow. (*fig:overview*)

</div>
<div id="div_fig2">

<img src="tmp_2301.06575/./plots/consistency_wlfixed.png" alt="Fig2" width="100%"/>

**Figure 2. -** \textit{Top.} Posterior predictive distribution $p(f(\theta)|x_\text{obs})$ of noiseless spectra (without the instrumental noise disturbance $\epsilon$) for the $99.7\%$, $95\%$ and $68.7\%$ quartiles (hues of blue), overlaid on top of the noiseless observed spectrum $f(\theta_\text{obs})$(black line). {\textit{Bottom.} Residuals of the posterior predictive samples, normalized by the standard deviation of the noise distribution for each spectral channel.
    } (*fig:consistency*)

</div>
<div id="div_fig3">

<img src="tmp_2301.06575/./plots/output-figure0.png" alt="Fig5" width="100%"/>

**Figure 5. -** Benchmark retrieval using neural posterior estimation. The corner plot shows 1d and 2d marginal posterior distributions obtained for the benchmark spectrum $x_\text{obs}$ for NPE (in blue) and for nested sampling (in orange). We observe that the nominal parameter values $\theta_\text{obs}$(in black) are well identified. The top right figure illustrates the posterior distribution of the P-T profiles.  (*fig:results*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\cyan}[1]{\textcolor{cyan}{#1}}$
$\newcommand{\magenta}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\arraystretch}{1.4}$
$\newcommand{\footnoterule}$
$\newcommand{\arraystretch}{1.35}$
$\newcommand{\hii}{H {\sc ii}}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\xsh}{X-shooter}$
$\newcommand{\prodimo}{{\sc ProDiMo}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\blue$}[1]{\textcolor{blue}{#1}}$
$\newcommand{$\red$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\cyan$}[1]{\textcolor{cyan}{#1}}$
$\newcommand{$\magenta$}[1]{\textcolor{magenta}{#1}}$
$\newcommand{$\arraystretch$}{1.4}$
$\newcommand{$\footnoterule$}$
$\newcommand{$\arraystretch$}{1.35}$
$\newcommand{$\hii$}{H {\sc ii}}$
$\newcommand{$\Msun$}{M_\odot}$
$\newcommand{$\Lsun$}{L_\odot}$
$\newcommand{$\xsh$}{X-shooter}$
$\newcommand{$\prodimo$}{{\sc ProDiMo}}$</div>



<div id="title">

# Massive pre-main-sequence stars in M17: Modelling hydrogen and dust in MYSO disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.06819-b31b1b.svg)](https://arxiv.org/abs/2301.06819)<mark>Appeared on: 2023-01-18</mark> - _18 pages, 16 figures. Accepted for publication in A&A_

</div>
<div id="authors">

<mark><mark>Frank Backs</mark></mark>, et al.

</div>
<div id="abstract">

**Abstract:** The young massive-star-forming region M17 contains optically visible massive pre-main-sequence stars that are surrounded by circumstellar disks. Such disks are expected to disappear when these stars enter the main sequence. The physical and dynamical structure of these remnant disks are poorly constrained, especially the inner regions where accretion, photo-evaporation, and companion formation and migration may be ongoing.We aim to constrain the physical properties of the inner parts of the circumstellar disks of massive young stellar objects B243 (6 M$_\odot$) and B331 (12 M$_\odot$), two systems for which the central star has been detected and characterized previously despite strong dust extinction.Two-dimensional radiation thermo-chemical modelling with$\prodimo$of double-peaked hydrogen lines of the Paschen and Brackett series observed with$\xsh$was used to probe the properties of the inner disk of the target sources. The model was modified to treat these lines. Additionally, the dust structure was studied by fitting the optical and near-infrared spectral energy distribution.B243 features a hot gaseous inner disk with dust at the sublimation radius at$\sim$3 AU. The disk appears truncated at roughly 6.5 AU; a cool outer disk of gas and dust may be present, but it cannot be detected with our data. B331 also has a hot gaseous inner disk. A gap separates the inner disk from a colder dusty outer disk starting at up to$\sim$100 AU. In both sources the inner disk extends to almost the stellar surface. Chemistry is essential for the ionization of hydrogen in these disks.The lack of a gap between the central objects and these disks suggests that they accrete through boundary-layer accretion. This would exclude the stars having a strong magnetic field.   Their structures suggest that both disks are transitional in nature, that is to say they are in the process of being cleared, either through boundary-layer accretion, photo-evaporation, or through companion activity.

</div>

<div id="div_fig1">

<img src="tmp_2301.06819/./parameters_per_line.png" alt="Fig5" width="100%"/>

**Figure 5. -** Parameter values of the inner disk with their uncertainties for each modelled line of B243 and B331 as function of their oscillator strength. The horizontal dashed lines indicate the best fit parameters for B243 when fitting Pa-9 and Pa-11 to Pa-16 simultaneously and for B331 when fitting Pa-9, Pa-11 to Pa-16 and Br-11 to Br-16. The corresponding line profiles are shown in Fig. \ref{p1:fig:B243_individual_line_fits} and Fig. \ref{p1:fig:B331_individual_line_fits} for B243 and B331, respectively. For each fit an inclination of 75$^\circ$ is used.
    \label{p1:fig:B243_B331_parameter_trend} (*p1:fig:B243_B331_parameter_trend*)

</div>
<div id="div_fig2">

<img src="tmp_2301.06819/./B243_line_profile_overview.png" alt="Fig13" width="100%"/>

**Figure 13. -** Best fitting models in blue to the observed lines in black for B243. The blue shaded regions indicate models that fall within the 1 $\sigma$ confidence interval. Each line is fitted individually with the inclination set to $75^\circ$, the respective parameters are shown in Fig. \ref{p1:fig:B243_B331_parameter_trend}. The orange profiles indicate the best fit when fitting Pa-16 to Pa-9 simultaneously. Notice the factor $\sim$30 range in line strengths covered by the Paschen series.
    \label{p1:fig:B243_individual_line_fits} (*p1:fig:B243_individual_line_fits*)

</div>
<div id="div_fig3">

<img src="tmp_2301.06819/./B331_line_profile_overview.png" alt="Fig14" width="100%"/>

**Figure 14. -** Best fitting models in blue to the observed lines in black for B331. The blue shaded regions indicate models that fall within the 1 $\sigma$ confidence interval. Each line is fitted individually with the inclination fixed at 75$^\circ$, the respective parameters are shown in Fig. \ref{p1:fig:B243_B331_parameter_trend}. The orange profiles indicate the best fit when fitting Pa-16 to Pa-9 and Br-16 to Br-11 simultaneously. Notice the factor $\sim$20 range in line strengths covered by the Paschen series. (*p1:fig:B331_individual_line_fits*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\tstar}{GJ~806\xspace}$
$\newcommand{\tplanet}{GJ~806 b\xspace}$
$\newcommand{\pytransit}{\textsc{PyTransit}\xspace}$
$\newcommand{\ldtk}{\textsc{LDTk}\xspace}$
$\newcommand{\lh}{\ensuremath{\ell}}$
$\newcommand{\llh}{\ensuremath{\ln P}}$
$\newcommand{\iid}{i.i.d.\xspace}$
$\newcommand{\pvec}{\ensuremath{\boldsymbol{\theta}}\xspace}$
$\newcommand{\covmat}{\emth{\boldsymbol{\Sigma}}}$
$\newcommand{\cmat}{\ensuremath{\mathbf{K}}\xspace}$
$\newcommand{\flux}{\ensuremath{\mathbf{f}}\xspace}$
$\newcommand{\gcm}{\emth{\mathrm{g cm^{-3}}}}$
$\newcommand{\smass}{\emth{\mathrm{M_\star}}}$
$\newcommand{\srad}{\emth{\mathrm{R_\star}}}$
$\newcommand{\pmass}{\emth{\mathrm{M_P}}}$
$\newcommand{\prad}{\emth{\mathrm{R_P}}}$
$\newcommand{\rptrue}{\ensuremath{r_\mathrm{p,true}}\xspace}$
$\newcommand{\ktrue}{\ensuremath{k_\mathrm{true}}\xspace}$
$\newcommand{\kapp}{\ensuremath{k_\mathrm{app}}\xspace}$
$\newcommand{\teff}{\ensuremath{T_\mathrm{eff}}\xspace}$
$\newcommand{\teffh}{\ensuremath{T_\mathrm{eff,H}}\xspace}$
$\newcommand{\teffc}{\ensuremath{T_\mathrm{eff,C}}\xspace}$
$\newcommand{\pzep}{\ensuremath{t_\mathrm{0}}\xspace}$
$\newcommand{\pper}{\ensuremath{p}\xspace}$
$\newcommand{\prho}{\ensuremath{\rho_\star}\xspace}$
$\newcommand{\pimp}{\ensuremath{b}\xspace}$
$\newcommand{\psma}{\ensuremath{a}\xspace}$
$\newcommand{\pcref}{\ensuremath{c}\xspace}$
$\newcommand{\para}{\ensuremath{k_\mathrm{app}}\xspace}$
$\newcommand{\ptra}{\ensuremath{k_\mathrm{true}}\xspace}$
$\newcommand{\mjup}{\ensuremath{M_\mathrm{Jup}}\xspace}$
$\newcommand{\rjup}{\ensuremath{R_\mathrm{Jup}}\xspace}$
$\newcommand{\george}{\textsc{George}\xspace}$
$\newcommand{\ud}{\ensuremath{\mathrm{d}\xspace}}$
$\newcommand{\plato}{\textit{PLATO}\xspace}$
$\newcommand{\tess}{\textit{TESS}\xspace}$
$\newcommand{\corot}{\textit{CoRoT}\xspace}$
$\newcommand{\kepler}{\textit{Kepler}\xspace}$
$\newcommand{\ktwo}{\textit{K2}\xspace}$
$\newcommand{\tmodel}{\ensuremath{\mathcal{T}}\xspace}$
$\newcommand{\UP}[1]{\ensuremath{\mathcal{U}(#1)}\xspace}$
$\newcommand{\NP}[1]{\ensuremath{\mathcal{N}(#1)}\xspace}$
$\newcommand{\emth}[1]{\ensuremath{#1}\xspace}$
$\newcommand{\mlp}[1]{\textcolor{red}{#1}}$
$\newcommand{\mlpd}[2]{\sout{#1} \textcolor{red}{#2}}$
$\newcommand{\ikoma}{\textcolor{red}}$
$\newcommand{\ajm}[1]{\textcolor{green}{#1}}$
$\newcommand{\Msun}{\hbox{M_{\odot}}}$
$\newcommand{\Rsun}{\hbox{R_{\odot}}}$
$\newcommand{\Mjup}{\hbox{M_{\rm Jup}}}$
$\newcommand{\Rjup}{\hbox{R_{\rm Jup}}}$
$\newcommand{\Mearth}{\hbox{M_{\oplus}}}$
$\newcommand{\Rearth}{\hbox{R_{\oplus}}}$
$\newcommand{\Mnep}{\hbox{M_{\rm Nep}}}$
$\newcommand{\hd20}{HD~209458 b}$
$\newcommand{\gj34}{GJ~3470 b}$
$\newcommand{\hat11}{HAT-P-11 b}$
$\newcommand{\hep}{He^{+}}$
$\newcommand{\hes}{He~(1^{1}S)}$
$\newcommand{\het}{He~(2^{3}S)}$
$\newcommand{\mlr}{\dot M}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\earth}{\oplus}$
$\newcommand{\halpha}{H\alpha }$
$\newcommand{\gs}{g s^{-1}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\rp}{R_{\rm P}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\tstar$}{GJ~806$\xspace$}$
$\newcommand{$\tplanet$}{GJ~806 b$\xspace$}$
$\newcommand{$\pytransit$}{$\textsc${PyTransit}$\xspace$}$
$\newcommand{$\ldtk$}{$\textsc${LDTk}$\xspace$}$
$\newcommand{$\lh$}{$\ensuremath${\ell}}$
$\newcommand{$\llh$}{$\ensuremath${\ln P}}$
$\newcommand{$\iid$}{i.i.d.$\xspace$}$
$\newcommand{$\pvec$}{$\ensuremath${\boldsymbol{\theta}}$\xspace$}$
$\newcommand{$\covmat$}{$\emth${\boldsymbol{\Sigma}}}$
$\newcommand{$\cmat$}{$\ensuremath${\mathbf{K}}$\xspace$}$
$\newcommand{$\flux$}{$\ensuremath${\mathbf{f}}$\xspace$}$
$\newcommand{$\gcm$}{$\emth${\mathrm{g cm^{-3}}}}$
$\newcommand{$\smass$}{$\emth${\mathrm{M_\star}}}$
$\newcommand{$\srad$}{$\emth${\mathrm{R_\star}}}$
$\newcommand{$\pmass$}{$\emth${\mathrm{M_P}}}$
$\newcommand{$\prad$}{$\emth${\mathrm{R_P}}}$
$\newcommand{$\rptrue$}{$\ensuremath${r_\mathrm{p,true}}$\xspace$}$
$\newcommand{$\ktrue$}{$\ensuremath${k_\mathrm{true}}$\xspace$}$
$\newcommand{$\kapp$}{$\ensuremath${k_\mathrm{app}}$\xspace$}$
$\newcommand{$\teff$}{$\ensuremath${T_\mathrm{eff}}$\xspace$}$
$\newcommand{$\teff$h}{$\ensuremath${T_\mathrm{eff,H}}$\xspace$}$
$\newcommand{$\teff$c}{$\ensuremath${T_\mathrm{eff,C}}$\xspace$}$
$\newcommand{$\pzep$}{$\ensuremath${t_\mathrm{0}}$\xspace$}$
$\newcommand{$\pper$}{$\ensuremath${p}$\xspace$}$
$\newcommand{$\prho$}{$\ensuremath${\rho_\star}$\xspace$}$
$\newcommand{$\pimp$}{$\ensuremath${b}$\xspace$}$
$\newcommand{$\psma$}{$\ensuremath${a}$\xspace$}$
$\newcommand{$\pcref$}{$\ensuremath${c}$\xspace$}$
$\newcommand{$\para$}{$\ensuremath${k_\mathrm{app}}$\xspace$}$
$\newcommand{$\ptra$}{$\ensuremath${k_\mathrm{true}}$\xspace$}$
$\newcommand{$\mjup$}{$\ensuremath${M_\mathrm{Jup}}$\xspace$}$
$\newcommand{$\rjup$}{$\ensuremath${R_\mathrm{Jup}}$\xspace$}$
$\newcommand{$\george$}{$\textsc${George}$\xspace$}$
$\newcommand{$\ud$}{$\ensuremath${\mathrm{d}$\xspace$}}$
$\newcommand{$\plato$}{\textit{PLATO}$\xspace$}$
$\newcommand{$\tess$}{\textit{TESS}$\xspace$}$
$\newcommand{$\corot$}{\textit{CoRoT}$\xspace$}$
$\newcommand{$\kepler$}{\textit{Kepler}$\xspace$}$
$\newcommand{$\ktwo$}{\textit{K2}$\xspace$}$
$\newcommand{$\tmodel$}{$\ensuremath${\mathcal{T}}$\xspace$}$
$\newcommand{$\UP$}[1]{$\ensuremath${\mathcal{U}(#1)}$\xspace$}$
$\newcommand{$\NP$}[1]{$\ensuremath${\mathcal{N}(#1)}$\xspace$}$
$\newcommand{$\emth$}[1]{$\ensuremath${#1}$\xspace$}$
$\newcommand{$\mlp$}[1]{\textcolor{red}{#1}}$
$\newcommand{$\mlp$d}[2]{\sout{#1} \textcolor{red}{#2}}$
$\newcommand{$\ikoma$}{\textcolor{red}}$
$\newcommand{$\ajm$}[1]{\textcolor{green}{#1}}$
$\newcommand{$\Msun$}{\hbox{M_{\odot}}}$
$\newcommand{$\Rsun$}{\hbox{R_{\odot}}}$
$\newcommand{$\Mjup$}{\hbox{M_{\rm Jup}}}$
$\newcommand{$\Rjup$}{\hbox{R_{\rm Jup}}}$
$\newcommand{$\Mearth$}{\hbox{M_{\oplus}}}$
$\newcommand{$\Rearth$}{\hbox{R_{\oplus}}}$
$\newcommand{$\Mnep$}{\hbox{M_{\rm Nep}}}$
$\newcommand{$\hd$20}{HD~209458 b}$
$\newcommand{$\gj$34}{GJ~3470 b}$
$\newcommand{$\hat$11}{HAT-P-11 b}$
$\newcommand{$\hep$}{He^{+}}$
$\newcommand{$\hes$}{He~(1^{1}S)}$
$\newcommand{$\het$}{He~(2^{3}S)}$
$\newcommand{$\mlr$}{\dot M}$
$\newcommand{$\lya$}{Ly\alpha}$
$\newcommand{$\earth$}{\oplus}$
$\newcommand{$\halpha$}{H\alpha }$
$\newcommand{$\gs$}{g s^{-1}}$
$\newcommand{$\kms$}{km s^{-1}}$
$\newcommand{$\rp$}{R_{\rm P}}$</div>



<div id="title">

# GJ 806 (TOI-4481): A bright nearby multi-planetary system with a transiting hot, low-density super-Earth

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.06873-b31b1b.svg)](https://arxiv.org/abs/2301.06873)<mark>Appeared on: 2023-01-18</mark> - _Under second review in A&A. This paper is NOT yet accepted, but it is made openly available to the community due to the approaching JWST deadline_

</div>
<div id="authors">

E. Palle, et al. -- incl., <mark><mark>M. Pérez-Torres</mark></mark>, <mark><mark>Th. Henning</mark></mark>, <mark><mark>E. Herrero</mark></mark>, <mark><mark>J. Lillo-Box</mark></mark>

</div>
<div id="abstract">

**Abstract:** One of the main scientific goals of the{\em TESS}mission is the discovery of transiting small planets around the closest and brightest stars in the sky.Here, using data from the CARMENES, MAROON-X, and HIRES spectrographs, together with{\em TESS}, we report the discovery and mass determination of a planetary system around the M1.5 V star GJ 806 (TOI-4481).GJ 806 is a bright ($V \approx$10.8 mag,$J \approx$7.3 mag) and nearby ($d$= 12 pc) M dwarf that hosts at least two planets.The innermost planet, GJ 806 b, is transiting and has an ultra-short orbital period of 0.93 d, a radius of$1.331\pm0.023 R_{\oplus}$, a mass of$1.90\pm 0.17 M_{\oplus}$, a mean density of$4.40 \pm 0.45$g cm$^{-3}$, and an equilibrium temperature of$940 \pm 10$K.We detect a second, non-transiting, super-Earth planet in the system, GJ 806 c, with an orbital period of 6.6 d, a minimum mass of$5.80 \pm 0.30 M_{\oplus}$, and an equilibrium temperature of$490 \pm 5$K. The radial velocity data also shows evidence for a third periodicity at 13.6 d, although the current dataset does not provide sufficient evidence to unambiguously distinguish between a third super-Earth mass ($M \sin i = 8.50\pm 0.45 M_{\oplus}$) planet or stellar activity. Additionally, we report one transit observation of GJ 806 b taken with CARMENES in search for a possible extended atmosphere of H or He, but we can only place upper limits to its existence. This is not surprising as our evolutionary models support the idea that any possible primordial H/He atmosphere that GJ 806 b might have had, would long have been lost. However, GJ 806 b's bulk density makes it likely that the planet hosts some type of volatile atmosphere.In fact, with a transmission spectroscopy metrics (TSM) of 44 and an emission spectroscopy metrics (ESM) of 24, GJ 806 b is to date the third-ranked terrestrial planet around an M dwarf suitable for transmission spectroscopy studies using{\em JWST}, and the most promising terrestrial planet for emission spectroscopy studies. GJ 806b is also an excellent target for the detection ofradio emission via star-planet interactions.

</div>

<div id="div_fig1">

<img src="tmp_2301.06873/./figures/GLS_Instruments_referee.png" alt="Fig13.1" width="50%"/><img src="tmp_2301.06873/./figures/WF_Instruments_referee.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** Left: Generalized Lomb-Scargle (GLS) periodograms of the HIRES (top), CARMENES (middle) and the two MAROON-X channels (bottom) radial velocity time series. Vertical dotted lines mark the highest peaks at 13.6 d (magenta), 6.6 d (green) and 0.92 d (orange). The 10\%, 1\% and 0.1\% FAP levels are indicated by gray dotted, dash dotted, and dashed lines, respectively. Right: Window functions of the HIRES, CARMENES and MAROON-X channels time series. Vertical dotted lines mark the 13.6 d (magenta), 6.6 d (green) and 0.92 d (orange) periods.   (*fig:singleRvs*)

</div>
<div id="div_fig2">

<img src="tmp_2301.06873/./figures/plot_mr.png" alt="Fig17.1" width="50%"/><img src="tmp_2301.06873/./figures/plot_mr_turbet_v2.png" alt="Fig17.2" width="50%"/>

**Figure 17. -** Mass radius diagrams of all known planets (left) and ultra-short period planets only (right). Only planets with mass determination better than 30\% and radius determination better than 10\%, according to the TEPCat database \citep{Southworth2011} are shown. The orange dots represent the known planet orbiting around M dwarfs  ($T_{\rm eff} < 4000 \mathrm{K}$) and the grey dots represent those orbiting around other stellar types.
Overplotted are theoretical models for the planet's internal composition from \citet{Zeng2019}(left) and \citet{Turbet2020}(right). GJ 806b is marked with a red dot, while the radius band that GJ 806c occupies is shaded in red.
 (*fig:mrdiagram*)

</div>
<div id="div_fig3">

<img src="tmp_2301.06873/./figures/ACTIVITY_Desembre.png" alt="Fig2" width="100%"/>

**Figure 2. -** Generalized Lomb-Scargle (GLS) periodograms of the activity indices derived from \texttt{SERVAL} for the CARMENES data. In all panels, the broken magenta lines indicate the planetary periodicities of 0.96, 6.6 and 13.6 days, and the broken red lines mark the 19.9, 34.6 and 48.1 day possible rotation periods discussed here. In the top two panels, the GLS for the CRX and dLW indices, are given independently for the visible and infrared channel spectra. The 10\%, 1\% and 0.1\% FAP levels are indicated by gray dotted, dash dotted, and dashed horizontal lines, respectively. (*fig:actindex*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

172  publications in the last 7 days.
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers